In [1]:
ROOT = './ODIR-5K'

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision
from sklearn.metrics import roc_auc_score
import os
from PIL import Image
import matplotlib.pyplot as plt
from natsort import natsorted

BATCH_SIZE = 32
IMG_HEIGHT = 250
IMG_WIDTH = 250
CLASS_NAMES = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = ImageFolder(root=ROOT+'/Training_Images/', transform=transform)
# validation_set = ImageFolder(root=ROOT+'/Validation_Images/', transform=transform)
test_set = ImageFolder(root=ROOT+'/Testing_Images/', transform=transform)

In [8]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size = 32,
    shuffle = True,
    num_workers = 2
)

# validation_loader = torch.utils.data.DataLoader(
#     validation_set,
#     batch_size = 16,
#     shuffle = False,
#     num_workers = 2
# )

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size = 16,
    shuffle = False,
    num_workers = 2
)

In [10]:
def train_model(model, num_epochs, criterion, optimizer, results_path):
    model.train()
    train_losses = np.zeros(num_epochs)
    val_losses = np.zeros(num_epochs)
    train_accracy = np.zeros(num_epochs)
    val_accracy = np.zeros(num_epochs)

    for epoch in range(num_epochs):
        # trainning
        running_loss = 0.0
        n = 0
        total=0
        correct=0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += 1
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)    # add in the number of labels in this minibatch
            correct += (predicted == labels).sum().item()  # add in the number of correct labels
        train_losses[epoch] = running_loss / n
        train_accracy[epoch]=correct/total
        # validation
#         running_loss = 0.0
#         n = 0
#         total=0
#         correct=0
#         with torch.no_grad():
#             for images,labels in validation_loader:
#                 images, labels = images.to(device), labels.to(device)
#                 outputs=model(images)
#                 running_loss += criterion(outputs,labels).item()
#                 n += 1
#                 _, predicted = torch.max(outputs.data, 1)
#                 total += labels.size(0)    # add in the number of labels in this minibatch
#                 correct += (predicted == labels).sum().item()  # add in the number of correct labels
#         val_losses[epoch]=running_loss/n
#         val_accracy[epoch]=correct/total

        print(f'Epoch [{epoch + 1}/{num_epochs}], training loss: {train_losses[epoch] : .3f} training accuracy: {train_accracy[epoch]: .1%}')

    torch.save({"state_dict": model.state_dict(), "train_losses": train_losses, "train_accracy": train_accracy}, results_path)


In [10]:
model_vgg16 = torchvision.models.vgg16(pretrained=True)
in_features = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(in_features, len(CLASS_NAMES), True)
model_vgg16 = model_vgg16.to(device)

result_path = ROOT+'/results/model_vgg_pretrained.pt'    
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_vgg16.parameters(), lr=0.01)
num_epochs = 20

train_model(model_vgg16, num_epochs, criterion, optimizer, result_path)

Epoch [1/20], training loss:  12271760.200 training accuracy:  14.4%
Epoch [2/20], training loss:  5.081 training accuracy:  15.0%
Epoch [3/20], training loss:  3.949 training accuracy:  15.0%
Epoch [4/20], training loss:  4.579 training accuracy:  14.8%
Epoch [5/20], training loss:  4.304 training accuracy:  15.1%
Epoch [6/20], training loss:  22622.371 training accuracy:  14.8%
Epoch [7/20], training loss:  11687.409 training accuracy:  14.5%
Epoch [8/20], training loss:  144.535 training accuracy:  15.1%
Epoch [9/20], training loss:  8.336 training accuracy:  15.1%
Epoch [10/20], training loss:  2.720 training accuracy:  15.0%
Epoch [11/20], training loss:  3.405 training accuracy:  14.8%
Epoch [12/20], training loss:  2.177 training accuracy:  15.0%
Epoch [13/20], training loss:  3.386 training accuracy:  14.7%
Epoch [14/20], training loss:  3.870 training accuracy:  15.1%
Epoch [15/20], training loss:  2.597 training accuracy:  15.2%
Epoch [16/20], training loss:  2.145 training a

In [11]:
resnet18 = torchvision.models.resnet18(pretrained=True)
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, len(CLASS_NAMES), True)
resnet50 = resnet18.to(device)

result_path = ROOT+'/results/model_resnet18_pretrained.pt'
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.01)
num_epochs = 20

train_model(resnet18, num_epochs, criterion, optimizer, result_path)

Epoch [1/20], training loss:  1.941 training accuracy:  26.0%
Epoch [2/20], training loss:  1.650 training accuracy:  35.7%
Epoch [3/20], training loss:  1.426 training accuracy:  43.7%
Epoch [4/20], training loss:  1.289 training accuracy:  49.4%
Epoch [5/20], training loss:  1.232 training accuracy:  51.3%
Epoch [6/20], training loss:  1.152 training accuracy:  54.1%
Epoch [7/20], training loss:  1.122 training accuracy:  55.7%
Epoch [8/20], training loss:  1.042 training accuracy:  58.4%
Epoch [9/20], training loss:  0.969 training accuracy:  61.8%
Epoch [10/20], training loss:  0.913 training accuracy:  64.2%
Epoch [11/20], training loss:  0.849 training accuracy:  67.1%
Epoch [12/20], training loss:  0.786 training accuracy:  69.0%
Epoch [13/20], training loss:  0.727 training accuracy:  71.1%
Epoch [14/20], training loss:  0.671 training accuracy:  73.2%
Epoch [15/20], training loss:  0.614 training accuracy:  74.9%
Epoch [16/20], training loss:  0.573 training accuracy:  76.5%
E

In [15]:
def test_model(model, path):
    d = torch.load(path)
    model.load_state_dict(d["state_dict"]) 
    model = model.to(device)
    total = 0
    correct = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    prob_all = []
    label_all = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Move predicted probabilities and labels to CPU and append to lists
            prob_all.extend(outputs.cpu().detach().numpy())
            label_all.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            true_positives += ((predicted == 1) & (labels == 1)).sum().item()
            false_positives += ((predicted == 1) & (labels == 0)).sum().item()
            false_negatives += ((predicted == 0) & (labels == 1)).sum().item()

    accuracy = correct / total
    if (true_positives + false_positives) != 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0.0  # or any other suitable value

    if (true_positives + false_negatives) != 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0.0  # or any other suitable value


    # Convert the lists to numpy arrays
    label_all = np.array(label_all)
    prob_all = np.array(prob_all)
    
    prob_all = np.exp(prob_all) / np.sum(np.exp(prob_all), axis=1, keepdims=True)
    
    label_all = label_all.reshape(-1, 1)  # Reshape label_all to (1800, 1)
    prob_all = prob_all.reshape(-1, 8)    # Reshape prob_all to (1800, 8

#     AUC = roc_auc_score(label_all, prob_all, multi_class='ovr')
    
    return accuracy, precision, recall


In [16]:
model_vgg16 = torchvision.models.vgg16(pretrained=False)
in_features = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(in_features, len(CLASS_NAMES), True)

results_path = ROOT + '/results/model_vgg_pretrained.pt'

accuracy, precision, recall = test_model(model_vgg16, results_path)

print('VGG16:')
print('  accuracy: {accuracy:.2f}'.format(accuracy=accuracy))
print('  precision: {precision:.2f}'.format(precision=precision))
print('  recall: {recall:.2f}'.format(recall=recall))
# print('  AUC: {AUC:.2f}'.format(AUC=AUC))

VGG16:
  accuracy: 0.46
  precision: 0.00
  recall: 0.00


In [17]:
resnet18 = torchvision.models.resnet18(pretrained=False)
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, len(CLASS_NAMES), True)

results_path = ROOT + '/results/model_resnet18_pretrained.pt'

accuracy, precision, recall = test_model(resnet18, results_path)

print('ResNet18:')
print('  accuracy: {accuracy:.2f}'.format(accuracy=accuracy))
print('  precision: {precision:.2f}'.format(precision=precision))
print('  recall: {recall:.2f}'.format(recall=recall))
# print('  AUC: {AUC:.2f}'.format(AUC=AUC))

ResNet18:
  accuracy: 0.26
  precision: 0.83
  recall: 0.71
